In [1]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# URL del archivo en el bucket público de AWS
url = "https://hybridge-education-machine-learning-datasets.s3.us-east-1.amazonaws.com/Fraud.csv"
df = pd.read_csv(url)

In [3]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
len(df)

6362620

# ¡¡¡6,362,620 OBSERVACIONES!!!

In [ ]:
# --- 1. Exploración y Preprocesamiento de Datos ---

# Identificar el desbalance de clases (Análisis solicitado)
print("## Distribución de la Variable Objetivo (isFraud)")
print(df['isFraud'].value_counts())
print("\n---")


# Aplicar One-Hot Encoding a la columna 'type' y eliminar las columnas de identificación
# Usamos drop_first=True para evitar multicolinealidad
X = df.drop(['isFraud', 'nameOrig', 'nameDest'], axis=1)
y = df['isFraud']

X = pd.get_dummies(X, columns=['type'], drop_first=True)


# Escalamiento de Características con StandardScaler
from sklearn.preprocessing import StandardScaler

print("## ⚙️ Escalamiento de Características")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
print("Características escaladas con éxito.")
print("\n---")

# División del Conjunto de Datos
from sklearn.model_selection import train_test_split

# Dividir en 80% entrenamiento y 20% prueba. Usamos stratify para mantener la proporción de fraudes.
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Tamaño del set de entrenamiento: {X_train.shape[0]} observaciones")
print(f"Tamaño del set de prueba: {X_test.shape[0]} observaciones")
print("\n---")


# --- 2. Entrenamiento del Modelo de Regresión Logística ---

from sklearn.linear_model import LogisticRegression

print("## 🧠 Iniciando Entrenamiento del Modelo...")
# Inicializar el modelo con 'liblinear' para manejar grandes datasets
model = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')

# Entrenar el modelo
model.fit(X_train, y_train)

print("¡Modelo de Regresión Logística entrenado con éxito!")
print("\n---")


# --- 3. Evaluación del Desempeño ---

from sklearn.metrics import confusion_matrix, classification_report

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

print("## 📈 Evaluación del Desempeño")

# Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("\n### Matriz de Confusión:")
# Mostrar los resultados de forma clara:
# [TN FP]
# [FN TP]
print(f"[[{conf_matrix[0][0]} {conf_matrix[0][1]}]")
print(f" [{conf_matrix[1][0]}  {conf_matrix[1][1]}]]")


# Reporte de Clasificación
class_report = classification_report(y_test, y_pred)
print("\n### Reporte de Clasificación:")
print(class_report)


# NOTA: Usé 'class_weight="balanced"' en el modelo para mejorar el recall en la clase minoritaria (fraude).
# Esto penaliza al modelo cuando se equivoca en la predicción de la clase minoritaria.